## Load and read data to database

In [ ]:
import psycopg2
import sys
import os
import time

# Function to handle SQL execution with retries in case of connection issues
def execute_batch(cur, conn, command, statement_number, retries=3):
    for attempt in range(retries):
        try:
            start_time = time.time()
            cur.execute(command)  # Execute the SQL command
            end_time = time.time()
            print(f"Executed statement {statement_number} in {end_time - start_time:.2f} seconds")
            return  # Exit the function if successful
        except psycopg2.OperationalError as e:
            # Handle connection errors with retry logic
            print(f"OperationalError on statement {statement_number}: {e}")
            if attempt < retries - 1:
                print(f"Retrying statement {statement_number} (attempt {attempt + 1}/{retries})...")
                time.sleep(2)  # Wait before retrying
                conn.rollback()  # Roll back the current transaction before retrying
                cur = conn.cursor()  # Recreate the cursor to handle potential connection issues
            else:
                raise e  # If all retry attempts fail, raise the error
        except psycopg2.Error as e:
            # Handle any other psycopg2 errors
            print(f"Error executing command {statement_number}: {command[:100]}...")  # Print part of the SQL command
            print(f"Error message: {e}")
            conn.rollback()  # Roll back the current transaction in case of failure
            sys.exit(1)  # Exit the script if a fatal error occurs

def main():
    # Use the db url provided by Railway(removed for security reasons)
    db_url = ''

    # Path to the SQL file that contains the commands to be executed
    sql_file_path = 'f1db-sql-postgresql.sql'

    # Ensure the SQL file exists before proceeding
    if not os.path.isfile(sql_file_path):
        print(f"SQL file not found: {sql_file_path}")
        sys.exit(1)

    try:
        # Connect to the PostgreSQL database using the provided URL
        print("Connecting to the database using DATABASE_URL...")
        conn = psycopg2.connect(db_url)
        conn.autocommit = False  # Disable autocommit to use manual transaction handling
        cur = conn.cursor()  # Create a cursor to interact with the database
        print("Connection successful.")

        # Initialize counters for progress tracking
        statement_count = 0  # Count the number of SQL statements executed
        line_count = 0  # Count the number of lines read from the SQL file
        batch_size = 100  # Commit the transaction every 100 statements for efficiency
        start_time = time.time()

        # Open and read the SQL file, executing commands in chunks
        with open(sql_file_path, 'r', encoding='utf-8') as f:
            print(f"Reading and executing SQL file: {sql_file_path}")
            sql_statement = ''
            for line in f:
                line_count += 1

                # Skip over comments and empty lines in the SQL file
                if line.strip().startswith('--') or line.strip() == '':
                    continue

                # Accumulate SQL commands until a complete statement (ending with ';') is reached
                sql_statement += line

                # Once a full statement is accumulated, execute it
                if line.strip().endswith(';'):
                    statement_count += 1
                    execute_batch(cur, conn, sql_statement, statement_count)
                    sql_statement = ''  # Reset the statement buffer after execution

                    # Commit the transaction after every 100 statements
                    if statement_count % batch_size == 0:
                        conn.commit()
                        print(f"Committed batch of {batch_size} statements.")

            # If there are any leftover SQL commands after the loop, execute them
            if sql_statement.strip():
                statement_count += 1
                execute_batch(cur, conn, sql_statement, statement_count)

            # Final commit to ensure all statements are saved
            conn.commit()

        total_time = time.time() - start_time
        print(f"Data imported successfully. Executed {statement_count} statements in {total_time:.2f} seconds.")

        # Close the cursor and the connection to the database
        cur.close()
        conn.close()
        print("Database connection closed.")

    except psycopg2.Error as e:
        # Handle any database errors and roll back the transaction
        print(f"Database error: {e}")
        conn.rollback()  # Roll back the current transaction in case of error
        sys.exit(1)
    except Exception as e:
        # Catch any other exceptions and exit
        print(f"An error occurred: {e}")
        sys.exit(1)

if __name__ == '__main__':
    main()

Connecting to the database using public URL...
Connection successful.
Reading and executing SQL file: f1db-sql-postgresql.sql
Executed statement 1 in 0.37 seconds
Executed statement 2 in 0.18 seconds
Executed statement 3 in 0.18 seconds
Executed statement 4 in 0.18 seconds
Executed statement 5 in 0.18 seconds
Executed statement 6 in 0.17 seconds
Executed statement 7 in 0.18 seconds
Executed statement 8 in 0.17 seconds
Executed statement 9 in 0.18 seconds
Executed statement 10 in 0.17 seconds
Executed statement 11 in 0.18 seconds
Executed statement 12 in 0.17 seconds
Executed statement 13 in 0.18 seconds
Executed statement 14 in 0.17 seconds
Executed statement 15 in 0.18 seconds
Executed statement 16 in 0.17 seconds
Executed statement 17 in 0.23 seconds
Executed statement 18 in 0.17 seconds
Executed statement 19 in 0.18 seconds
Executed statement 20 in 0.18 seconds
Executed statement 21 in 0.18 seconds
Executed statement 22 in 0.17 seconds
Executed statement 23 in 0.18 seconds
Executed 